지하철 혼잡도

참조(외부) 자료

In [ ]:
1. 대중교통 혼잡도 (정성적 자료)
1.1 지하철
<출근길 붐비는 서울지하철 4호선…한 칸은 의자 없이>, 연합뉴스, https://www.youtube.com/watch?v=Bo9QPNEmZ6E
1.2 버스
"계속 늘어나는 광역버스로 혼잡해지는 서울 도심, 해결 방안은?", https://mediahub.seoul.go.kr/archives/2011154

